<a href="https://colab.research.google.com/github/kattens/PubChem-Data-Handler/blob/main/Visualization_and_Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

in this notebook we will use py3mol package to get access to the protein sequences and 3d structure, we will also do alignment of the structure of 2 proteins as well to check the similarities between them.

In [3]:
import requests
import json

# UniProt API endpoint for the P00533 entry
uniprot_id = "P00533"
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"

# Send a GET request to the UniProt API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Print the JSON to see its structure
    print(json.dumps(data, indent=2))
else:
    print(f"Failed to fetch data: {response.status_code}")


{
  "entryType": "UniProtKB reviewed (Swiss-Prot)",
  "primaryAccession": "P00533",
  "secondaryAccessions": [
    "O00688",
    "O00732",
    "P06268",
    "Q14225",
    "Q68GS5",
    "Q92795",
    "Q9BZS2",
    "Q9GZX1",
    "Q9H2C9",
    "Q9H3C9",
    "Q9UMD7",
    "Q9UMD8",
    "Q9UMG5"
  ],
  "uniProtkbId": "EGFR_HUMAN",
  "entryAudit": {
    "firstPublicDate": "1986-07-21",
    "lastAnnotationUpdateDate": "2025-02-05",
    "lastSequenceUpdateDate": "1997-11-01",
    "entryVersion": 296,
    "sequenceVersion": 2
  },
  "annotationScore": 5.0,
  "organism": {
    "scientificName": "Homo sapiens",
    "commonName": "Human",
    "taxonId": 9606,
    "lineage": [
      "Eukaryota",
      "Metazoa",
      "Chordata",
      "Craniata",
      "Vertebrata",
      "Euteleostomi",
      "Mammalia",
      "Eutheria",
      "Euarchontoglires",
      "Primates",
      "Haplorrhini",
      "Catarrhini",
      "Hominidae",
      "Homo"
    ]
  },
  "proteinExistence": "1: Evidence at protein lev

In [8]:
import requests
import json

# UniProt API endpoint for the P00533 entry
uniprot_id = "P00533"
url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}.json"

# Send a GET request to the UniProt API
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()

    # Extract PDB structure information from 'dbReferences'
    pdb_dict = {}
    for db_reference in data.get("dbReferences", []):
        if db_reference.get("type") == "PDB":
            pdb_code = db_reference.get("id")
            properties = {prop["key"]: prop["value"] for prop in db_reference.get("properties", [])}

            # Extract method, resolution, and chain information
            method = properties.get("Method", "N/A")
            resolution = properties.get("Resolution", "N/A")
            chains_info = properties.get("Chains", "N/A")

            # Parse chain and residue range if available
            chain_data = []
            if chains_info != "N/A":
                chains = chains_info.split(", ")
                for chain in chains:
                    if "=" in chain:
                        chain_id, range_info = chain.split("=")
                        start, finish = map(int, range_info.split("-"))
                        chain_data.append({"chain": chain_id, "start": start, "finish": finish})

            # Add parsed data to the dictionary
            pdb_dict[pdb_code] = {
                "method": method,
                "resolution": resolution,
                "chains": chain_data
            }

    # Display the resulting dictionary
    if pdb_dict:
        print(json.dumps(pdb_dict, indent=2))
    else:
        print("No PDB structure information found in 'dbReferences'.")
else:
    print(f"Failed to fetch data: {response.status_code}")


No PDB structure information found in 'dbReferences'.


In [10]:
import requests
import json

def get_pdb_info_from_uniprot(uniprot_id):
    """
    Retrieves PDB IDs associated with a UniProt ID from the UniProt API,
    and then retrieves information about each PDB entry using the RCSB PDB API.

    Args:
        uniprot_id (str): The UniProt ID (e.g., P0DTC2).

    Returns:
        dict: A dictionary where keys are PDB IDs and values are dictionaries
              containing information about the PDB entry.  Returns an empty
              dictionary if no PDB IDs are found or if there's an error.
    """

    # Step 1: Get PDB IDs associated with the UniProt ID from UniProt API
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}?format=json"

    try:
        response = requests.get(uniprot_url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        pdb_ids = []
        for dbReference in data.get('uniProtKBCrossReferences', []):
            if dbReference.get('database') == 'PDB':
                pdb_ids.append(dbReference['id'])

        if not pdb_ids:
            print(f"No PDB entries found for UniProt ID: {uniprot_id}")
            return {}

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from UniProt for {uniprot_id}: {e}")
        return {}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from UniProt for {uniprot_id}: {e}")
        return {}
    except KeyError as e:
        print(f"Error parsing UniProt JSON for {uniprot_id}: Missing key: {e}")
        return {}


    # Step 2: Get information about each PDB entry from RCSB PDB API
    pdb_info = {}
    for pdb_id in pdb_ids:
        pdb_url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"  # Use RCSB API for PDB details

        try:
            pdb_response = requests.get(pdb_url)
            pdb_response.raise_for_status() # Raise HTTPError for bad responses
            pdb_data = pdb_response.json()
            pdb_info[pdb_id] = pdb_data
        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Error: {e}"} # Store error information
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"JSON decode error: {e}"}
        except Exception as e:
            print(f"Unexpected error processing PDB ID {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Unexpected error: {e}"}


    return pdb_info


# Example usage:
uniprot_id = "P0DTC2"  # Example: Spike glycoprotein from SARS-CoV-2
pdb_data = get_pdb_info_from_uniprot(uniprot_id)

if pdb_data:
    for pdb_id, info in pdb_data.items():
        print(f"\nInformation for PDB ID: {pdb_id}")
        if "error" in info:
            print(f"  Error: {info['error']}")
        else:
            print(json.dumps(info, indent=2)) # Nicely formatted JSON output
else:
    print("No PDB information found.")

Streaming output truncated to the last 5000 lines.
    }
  ],
  "em_particle_selection": [
    {
      "id": "1",
      "image_processing_id": "1",
      "num_particles_selected": 4961369
    }
  ],
  "em_software": [
    {
      "category": "PARTICLE SELECTION",
      "id": "1",
      "image_processing_id": "1",
      "name": "cryoSPARC"
    },
    {
      "category": "IMAGE ACQUISITION",
      "id": "2",
      "imaging_id": "1"
    },
    {
      "category": "MASKING",
      "id": "3"
    },
    {
      "category": "CTF CORRECTION",
      "id": "4",
      "image_processing_id": "1"
    },
    {
      "category": "LAYERLINE INDEXING",
      "id": "5"
    },
    {
      "category": "DIFFRACTION INDEXING",
      "id": "6"
    },
    {
      "category": "MODEL FITTING",
      "fitting_id": "1",
      "id": "7"
    },
    {
      "category": "OTHER",
      "id": "8"
    },
    {
      "category": "MODEL REFINEMENT",
      "fitting_id": "1",
      "id": "9",
      "name": "PHENIX",
      "

In [11]:
import requests
import json

def get_pdb_info_from_uniprot(uniprot_id):
    """
    Retrieves PDB IDs associated with a UniProt ID from the UniProt API,
    and then retrieves specific information (resolution, length, etc.) about
    each PDB entry using the RCSB PDB API.

    Args:
        uniprot_id (str): The UniProt ID (e.g., P0DTC2).

    Returns:
        dict: A dictionary where keys are PDB IDs and values are dictionaries
              containing the extracted information (resolution, length, etc.).
              Returns an empty dictionary if no PDB IDs are found or if there's an error.
    """

    # Step 1: Get PDB IDs associated with the UniProt ID from UniProt API
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}?format=json"

    try:
        response = requests.get(uniprot_url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        pdb_ids = []
        for dbReference in data.get('uniProtKBCrossReferences', []):
            if dbReference.get('database') == 'PDB':
                pdb_ids.append(dbReference['id'])

        if not pdb_ids:
            print(f"No PDB entries found for UniProt ID: {uniprot_id}")
            return {}

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from UniProt for {uniprot_id}: {e}")
        return {}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from UniProt for {uniprot_id}: {e}")
        return {}
    except KeyError as e:
        print(f"Error parsing UniProt JSON for {uniprot_id}: Missing key: {e}")
        return {}


    # Step 2: Get information about each PDB entry from RCSB PDB API
    pdb_info = {}
    for pdb_id in pdb_ids:
        pdb_url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"  # Use RCSB API for PDB details

        try:
            pdb_response = requests.get(pdb_url)
            pdb_response.raise_for_status() # Raise HTTPError for bad responses
            pdb_data = pdb_response.json()

            # Extract the desired information.  Add more fields as needed.
            extracted_info = {
                "pdb_id": pdb_id,
                "resolution": None, # Initialize to None in case it's missing
                "length": None,
                "method": None,
                "chain_id": None,
                "entity_id": None,
                "seq_start": None,
                "seq_end": None,


                # Add more fields as needed. Find fields by inspecting the JSON
                # response from the RCSB PDB API (printed in previous example)
            }


            # Extract resolution
            if 'rcsb_entry_info' in pdb_data and 'resolution_combined' in pdb_data['rcsb_entry_info']:
                extracted_info["resolution"] = pdb_data['rcsb_entry_info']['resolution_combined'][0] # Taking the first resolution if there are multiple


            # Extract method
            if 'exptl' in pdb_data and pdb_data['exptl']:
                extracted_info['method'] = pdb_data['exptl'][0]['method']


            # Now we need to access the correct entity and chain
            if 'rcsb_entry_container_identifiers' in pdb_data:
               if pdb_data['rcsb_entry_container_identifiers']['entity_ids']:
                entity_id = pdb_data['rcsb_entry_container_identifiers']['entity_ids'][0]  #default the first one
                extracted_info['entity_id'] = entity_id

                # Extract chain ID and length associated with entity ID
                for entity in pdb_data.get("entities", []):
                    if entity['entity_id'] == entity_id:
                        extracted_info['length'] = entity['rcsb_entity_info']['entity_length']

                # Extract chain ID
                if 'rcsb_entry_container_identifiers' in pdb_data:
                   if pdb_data['rcsb_entry_container_identifiers']['assembly_ids']:
                       assembly_id = pdb_data['rcsb_entry_container_identifiers']['assembly_ids'][0]  #default the first one

                       for assembly in pdb_data.get("assemblies", []):
                            if assembly['assembly_id'] == assembly_id:
                                for chain in assembly.get("rcsb_assembly_container_identifiers", {}).get("auth_asym_ids",[]):
                                   extracted_info['chain_id'] = chain   #default the first one

                # Get the range of the protein
                for struct_ref in pdb_data.get("struct_ref", []):
                    for seq_range in struct_ref.get("seq_range", []):
                         extracted_info['seq_start'] = seq_range["beg_seq_id"]
                         extracted_info['seq_end'] = seq_range["end_seq_id"]


            pdb_info[pdb_id] = extracted_info


        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Error: {e}"} # Store error information
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"JSON decode error: {e}"}
        except Exception as e:
            print(f"Unexpected error processing PDB ID {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Unexpected error: {e}"}


    return pdb_info


# Example usage:
uniprot_id = "P0DTC2"  # Example: Spike glycoprotein from SARS-CoV-2
pdb_data = get_pdb_info_from_uniprot(uniprot_id)

if pdb_data:
    for pdb_id, info in pdb_data.items():
        print(f"\nInformation for PDB ID: {pdb_id}")
        if "error" in info:
            print(f"  Error: {info['error']}")
        else:
            print(json.dumps(info, indent=2)) # Nicely formatted JSON output
else:
    print("No PDB information found.")


Information for PDB ID: 6LVN
{
  "pdb_id": "6LVN",
  "resolution": 2.47,
  "length": null,
  "method": "X-RAY DIFFRACTION",
  "chain_id": null,
  "entity_id": "1",
  "seq_start": null,
  "seq_end": null
}

Information for PDB ID: 6LXT
{
  "pdb_id": "6LXT",
  "resolution": 2.9,
  "length": null,
  "method": "X-RAY DIFFRACTION",
  "chain_id": null,
  "entity_id": "1",
  "seq_start": null,
  "seq_end": null
}

Information for PDB ID: 6LZG
{
  "pdb_id": "6LZG",
  "resolution": 2.5,
  "length": null,
  "method": "X-RAY DIFFRACTION",
  "chain_id": null,
  "entity_id": "1",
  "seq_start": null,
  "seq_end": null
}

Information for PDB ID: 6M0J
{
  "pdb_id": "6M0J",
  "resolution": 2.45,
  "length": null,
  "method": "X-RAY DIFFRACTION",
  "chain_id": null,
  "entity_id": "1",
  "seq_start": null,
  "seq_end": null
}

Information for PDB ID: 6M17
{
  "pdb_id": "6M17",
  "resolution": 2.9,
  "length": null,
  "method": "ELECTRON MICROSCOPY",
  "chain_id": null,
  "entity_id": "1",
  "seq_start

In [15]:
import requests
import json

def get_pdb_info_from_uniprot(uniprot_id):
    """
    Retrieves PDB IDs associated with a UniProt ID from the UniProt API,
    and then retrieves specific information (resolution, length, etc.) about
    each PDB entry using the RCSB PDB API, formatting the output as a dictionary
    with the structure you specified.

    Args:
        uniprot_id (str): The UniProt ID (e.g., P0DTC2).

    Returns:
        dict: A dictionary with PDB IDs as keys and dictionaries of structured
              information as values, formatted as you requested.
              Returns an empty dictionary if no PDB IDs are found or if there's an error.
    """

    # Step 1: Get PDB IDs associated with the UniProt ID from UniProt API
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}?format=json"

    try:
        response = requests.get(uniprot_url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        pdb_ids = []
        for dbReference in data.get('uniProtKBCrossReferences', []):
            if dbReference.get('database') == 'PDB':
                pdb_ids.append(dbReference['id'])

        if not pdb_ids:
            print(f"No PDB entries found for UniProt ID: {uniprot_id}")
            return {}

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from UniProt for {uniprot_id}: {e}")
        return {}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from UniProt for {uniprot_id}: {e}")
        return {}
    except KeyError as e:
        print(f"Error parsing UniProt JSON for {uniprot_id}: Missing key: {e}")
        return {}


    # Step 2: Get information about each PDB entry from RCSB PDB API
    pdb_info = {}
    for pdb_id in pdb_ids:
        pdb_url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"  # Use RCSB API for PDB details

        try:
            pdb_response = requests.get(pdb_url)
            pdb_response.raise_for_status() # Raise HTTPError for bad responses
            pdb_data = pdb_response.json()


            # Initialize the dictionary for this PDB ID
            pdb_info[pdb_id] = {
                "method": None,
                "resolution": None,
                "chains": []  # Initialize the chains list
            }

            # Extract method
            if 'exptl' in pdb_data and pdb_data['exptl']:
                pdb_info[pdb_id]["method"] = pdb_data['exptl'][0]['method']

            # Extract resolution
            if 'rcsb_entry_info' in pdb_data and 'resolution_combined' in pdb_data['rcsb_entry_info']:
                pdb_info[pdb_id]["resolution"] = pdb_data['rcsb_entry_info']['resolution_combined'][0]

            # Extract chain information and sequence range
            chain_id = None
            seq_start = None
            seq_end = None

            if 'rcsb_entry_container_identifiers' in pdb_data:
               if pdb_data['rcsb_entry_container_identifiers']['entity_ids']:
                    entity_id = pdb_data['rcsb_entry_container_identifiers']['entity_ids'][0]

                    #Extract chain ID
                    if 'rcsb_entry_container_identifiers' in pdb_data:
                       if pdb_data['rcsb_entry_container_identifiers']['assembly_ids']:
                           assembly_id = pdb_data['rcsb_entry_container_identifiers']['assembly_ids'][0]  #default the first one

                           for assembly in pdb_data.get("assemblies", []):
                                if assembly['assembly_id'] == assembly_id:
                                    for chain in assembly.get("rcsb_assembly_container_identifiers", {}).get("auth_asym_ids",[]):
                                       chain_id = chain   #default the first one

                    # Get the range of the protein
                    for struct_ref in pdb_data.get("struct_ref", []):
                        for seq_range in struct_ref.get("seq_range", []):
                            seq_start = seq_range["beg_seq_id"]
                            seq_end = seq_range["end_seq_id"]

                    # Add the chain information to the chains list
                    if chain_id is not None and seq_start is not None and seq_end is not None:
                        pdb_info[pdb_id]["chains"].append({
                            "chain": chain_id,
                            "start": seq_start,
                            "finish": seq_end
                        })


        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Error: {e}"} # Store error information
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"JSON decode error: {e}"}
        except Exception as e:
            print(f"Unexpected error processing PDB ID {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Unexpected error: {e}"}


    return pdb_info


# Example usage:
uniprot_id = "P00533"  # Example: Spike glycoprotein from SARS-CoV-2
pdb_data = get_pdb_info_from_uniprot(uniprot_id)

if pdb_data:
    print(json.dumps(pdb_data, indent=4))  # Print the structured output
else:
    print("No PDB information found.")

{
    "1IVO": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 3.3,
        "chains": []
    },
    "1M14": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 2.6,
        "chains": []
    },
    "1M17": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 2.6,
        "chains": []
    },
    "1MOX": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 2.5,
        "chains": []
    },
    "1NQL": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 2.8,
        "chains": []
    },
    "1XKK": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 2.4,
        "chains": []
    },
    "1YY9": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 2.605,
        "chains": []
    },
    "1Z9I": {
        "method": "SOLUTION NMR",
        "resolution": null,
        "chains": []
    },
    "2EB2": {
        "method": "X-RAY DIFFRACTION",
        "resolution": 2.5,
        "chains": []
    },
    "2EB3": {
        "metho

In [16]:
# After the get_pdb_info_from_uniprot function and its usage:

def get_top_n_resolution(pdb_data, n=3):
    """
    Finds the top N PDB entries with the best (highest) resolution.

    Args:
        pdb_data (dict): The dictionary returned by get_pdb_info_from_uniprot.
        n (int): The number of top entries to return (default: 3).

    Returns:
        list: A list of tuples, where each tuple contains (PDB ID, resolution).
              Sorted by resolution in ascending order (best resolution first).
    """

    resolution_list = []
    for pdb_id, info in pdb_data.items():
        if "error" not in info and info["resolution"] is not None:
            try:
                resolution = info["resolution"]
                if isinstance(resolution, str):
                    resolution = float(resolution.replace("Å", ""))  # Remove Å and convert to float if it's a string
                elif isinstance(resolution, (int, float)):
                   pass  #It's already a number.
                else:
                    print(f"Warning: Unexpected resolution type for PDB ID: {pdb_id}, skipping.")
                    continue

                resolution_list.append((pdb_id, resolution))
            except ValueError:
                print(f"Warning: Could not convert resolution to float for PDB ID: {pdb_id}, skipping.")

    # Sort by resolution (lower is better)
    resolution_list.sort(key=lambda x: x[1])

    return resolution_list[:n]  # Return the top N


# Get the top 3 PDBs by resolution:
if pdb_data:  # Ensure pdb_data is not empty
    top_3 = get_top_n_resolution(pdb_data, n=3)

    print("\nTop 3 PDBs by Resolution:")
    for pdb_id, resolution in top_3:
        print(f"  {pdb_id}: {resolution:.2f} Å") # Format resolution nicely
else:
    print("No PDB data available to find top resolutions.")


Top 3 PDBs by Resolution:
  8A27: 1.07 Å
  8A2D: 1.11 Å
  5UG9: 1.33 Å


In [17]:
!pip install biopython
!pip install py3Dmol #for visualization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.8 MB/s eta 0:00:00


In [18]:
#We will import all the libraries here but also will add each one in the related block too.
import requests
from Bio.PDB import PDBParser,PPBuilder, PDBList,Superimposer, PDBIO, Select
import py3Dmol

In [19]:
!wget -O 8A27.pdb https://files.rcsb.org/download/8A27.pdb
!wget -O 5UG9.pdb https://files.rcsb.org/download/5UG9.pdb

--2025-03-03 22:56:54--  https://files.rcsb.org/download/8A27.pdb
Resolving files.rcsb.org (files.rcsb.org)... 3.166.135.84, 3.166.135.85, 3.166.135.67, ...
Connecting to files.rcsb.org (files.rcsb.org)|3.166.135.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘8A27.pdb’

8A27.pdb                [ <=>                ] 482.92K  --.-KB/s    in 0.06s   

2025-03-03 22:56:55 (7.51 MB/s) - ‘8A27.pdb’ saved [494505]

--2025-03-03 22:56:55--  https://files.rcsb.org/download/5UG9.pdb
Resolving files.rcsb.org (files.rcsb.org)... 3.166.135.84, 3.166.135.85, 3.166.135.67, ...
Connecting to files.rcsb.org (files.rcsb.org)|3.166.135.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘5UG9.pdb’

5UG9.pdb                [ <=>                ] 255.10K  --.-KB/s    in 0.05s   

2025-03-03 22:56:56 (4.57 MB/s) - ‘5UG9.pdb’ saved [261225]



In [20]:
import py3Dmol

pdb_id = "8A27"  # same example as part 2
view = py3Dmol.view(query=f'pdb:{pdb_id}', options={'doAssembly': True})
view.setStyle({'cartoon': {'color': 'spectrum'}})
view.addStyle({'model': -1}, {'stick': {}})
view.zoomTo()
view.show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [22]:
import requests
import json

def get_sequence_and_pdb_by_taxonomy(taxonomy_id):
    """
    Retrieves UniProt IDs associated with a given taxonomy ID, then fetches
    the amino acid sequence and PDB information for each UniProt entry.

    Args:
        taxonomy_id (str or int): The NCBI taxonomy ID (e.g., 5833).

    Returns:
        dict: A dictionary with UniProt IDs as keys. Each UniProt ID maps
              to a dictionary containing "sequence" (amino acid sequence) and
              "pdb_info" (PDB information from the RCSB API, structured as before).
              Returns an empty dictionary if no UniProt IDs are found for the
              taxonomy ID or if there are any errors.
    """

    # Step 1: Get UniProt IDs associated with the taxonomy ID using UniProt API
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/stream?query=taxonomy_id:{taxonomy_id}&format=json&fields=accession,sequence"

    try:
        response = requests.get(uniprot_url)
        response.raise_for_status()
        data = response.json()

        if not data:
            print(f"No UniProt entries found for taxonomy ID: {taxonomy_id}")
            return {}

        results = {}
        if isinstance(data, dict): #if there are multiple data it is stored in dict
            data = [data] #Put it in a list so the following logic is the same
        #The logic is the same for the next part of getting the result and it is handle the same
        for entry in data:
            uniprot_id = entry.get("primaryAccession")
            sequence = entry.get("sequence", {}).get("value")

            if not uniprot_id:
                print(f"Warning: No UniProt ID found for an entry with taxonomy ID: {taxonomy_id}")
                continue

            if not sequence:
                print(f"Warning: No sequence found for UniProt ID: {uniprot_id}")
                #continue #If no sequence is found, still try getting PDB information

            results[uniprot_id] = {"sequence": sequence, "pdb_info": {}}
            # Initialize pdb_info.  We'll populate it later.


    except requests.exceptions.RequestException as e:
        print(f"Error fetching UniProt IDs for taxonomy ID {taxonomy_id}: {e}")
        return {}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from UniProt for taxonomy ID {taxonomy_id}: {e}")
        return {}
    except KeyError as e:
        print(f"Error parsing UniProt JSON for taxonomy ID {taxonomy_id}: Missing key: {e}")
        return {}

    # Step 2: Get PDB information for each UniProt ID (using the existing function)
    for uniprot_id in results:
        results[uniprot_id]["pdb_info"] = get_pdb_info_from_uniprot(uniprot_id)

    return results



def get_pdb_info_from_uniprot(uniprot_id):
    """
    Retrieves PDB IDs associated with a UniProt ID from the UniProt API,
    and then retrieves specific information (resolution, length, etc.) about
    each PDB entry using the RCSB PDB API, formatting the output as a dictionary
    with the structure you specified.

    Args:
        uniprot_id (str): The UniProt ID (e.g., P0DTC2).

    Returns:
        dict: A dictionary with PDB IDs as keys and dictionaries of structured
              information as values, formatted as you requested.
              Returns an empty dictionary if no PDB IDs are found for the
              taxonomy ID or if there are any errors.
    """

    # Step 1: Get PDB IDs associated with the UniProt ID from UniProt API
    uniprot_url = f"https://rest.uniprot.org/uniprotkb/{uniprot_id}?format=json"

    try:
        response = requests.get(uniprot_url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        pdb_ids = []
        for dbReference in data.get('uniProtKBCrossReferences', []):
            if dbReference.get('database') == 'PDB':
                pdb_ids.append(dbReference['id'])

        if not pdb_ids:
            print(f"No PDB entries found for UniProt ID: {uniprot_id}")
            return {}

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from UniProt for {uniprot_id}: {e}")
        return {}
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from UniProt for {uniprot_id}: {e}")
        return {}
    except KeyError as e:
        print(f"Error parsing UniProt JSON for {uniprot_id}: Missing key: {e}")
        return {}


    # Step 2: Get PDB information for each UniProt ID (using the existing function)
    pdb_info = {}
    for pdb_id in pdb_ids:
        pdb_url = f"https://data.rcsb.org/rest/v1/core/entry/{pdb_id}"  # Use RCSB API for PDB details

        try:
            pdb_response = requests.get(pdb_url)
            pdb_response.raise_for_status() # Raise HTTPError for bad responses
            pdb_data = pdb_response.json()


            # Initialize the dictionary for this PDB ID
            pdb_info[pdb_id] = {
                "method": None,
                "resolution": None,
                "chains": []  # Initialize the chains list
            }

            # Extract method
            if 'exptl' in pdb_data and pdb_data['exptl']:
                pdb_info[pdb_id]["method"] = pdb_data['exptl'][0]['method']

            # Extract resolution
            if 'rcsb_entry_info' in pdb_data and 'resolution_combined' in pdb_data['rcsb_entry_info']:
                pdb_info[pdb_id]["resolution"] = pdb_data['rcsb_entry_info']['resolution_combined'][0]

            # Extract chain information and sequence range
            chain_id = None
            seq_start = None
            seq_end = None

            if 'rcsb_entry_container_identifiers' in pdb_data:
               if pdb_data['rcsb_entry_container_identifiers']['entity_ids']:
                    entity_id = pdb_data['rcsb_entry_container_identifiers']['entity_ids'][0]

                    #Extract chain ID
                    if 'rcsb_entry_container_identifiers' in pdb_data:
                       if pdb_data['rcsb_entry_container_identifiers']['assembly_ids']:
                           assembly_id = pdb_data['rcsb_entry_container_identifiers']['assembly_ids'][0]  #default the first one

                           for assembly in pdb_data.get("assemblies", []):
                                if assembly['assembly_id'] == assembly_id:
                                    for chain in assembly.get("rcsb_assembly_container_identifiers", {}).get("auth_asym_ids",[]):
                                       chain_id = chain   #default the first one

                    # Get the range of the protein
                    for struct_ref in pdb_data.get("struct_ref", []):
                        for seq_range in struct_ref.get("seq_range", []):
                            seq_start = seq_range["beg_seq_id"]
                            seq_end = seq_range["end_seq_id"]

                    # Add the chain information to the chains list
                    if chain_id is not None and seq_start is not None and seq_end is not None:
                        pdb_info[pdb_id]["chains"].append({
                            "chain": chain_id,
                            "start": seq_start,
                            "finish": seq_end
                        })


        except requests.exceptions.RequestException as e:
            print(f"Error fetching data from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Error: {e}"} # Store error information
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from RCSB PDB for {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"JSON decode error: {e}"}
        except Exception as e:
            print(f"Unexpected error processing PDB ID {pdb_id}: {e}")
            pdb_info[pdb_id] = {"error": f"Unexpected error: {e}"}


    return pdb_info


# Example usage:
taxonomy_id = 5833  # Example: Homo sapiens
#taxonomy_id = 36329  # Example: Plasmodium falciparum
taxonomy_data = get_sequence_and_pdb_by_taxonomy(taxonomy_id)


if taxonomy_data:
    print(json.dumps(taxonomy_data, indent=4))  # Print the structured output
else:
    print("No data found for the given taxonomy ID.")

No data found for the given taxonomy ID.


In [ ]:
taxonomy_id = 5833  # Example: Homo sapiens

In [ ]:
taxonomy_id = 36329  # Example: Plasmodium falciparum